In [1]:
import openpyxl
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import basedosdados as bd
import numpy as np


In [2]:
os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\MUNIC")

In [3]:
df18=pd.read_csv("MUNIC_educacao_2018.csv", sep=';')
df21=pd.read_csv("MUNIC_educacao_2021.csv", sep=';')

In [4]:
df18 = df18[['Cod Municipio', 'genero', 'idade', 'raca_cor',
        'escolaridade']]
df21 = df21[['CodMun', 'genero', 'idade', 'raca_cor',
        'escolaridade']]
        

In [5]:
df18['ano']=2018
df21['ano']=2021

In [6]:
df18 = df18.rename(columns={'Cod Municipio':'id_municipio','CodMun':'id_municipio'})
df21 = df21.rename(columns={'Cod Municipio':'id_municipio','CodMun':'id_municipio'})

In [8]:
df = pd.concat([df18,df21])

In [9]:
df['idade']=np.where(df['idade']=='(*) Não soube informar',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='Recusa',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='Não informou',np.nan,df['idade'])
df['idade'] =pd.to_numeric(df['idade'])

In [10]:
limites = [0, 30, 50,65,100]
categorias = ['Entre 18-29', 'Entre 30-49', 'Entre 50-64', 'Acima de 65']

df['faixa_etaria'] = pd.cut(df['idade'], bins=limites, labels=categorias)


In [11]:
## criando faixa etária
df1= df

In [13]:
### Dataframe do mapa

df1 = df1[['ano','id_municipio', 'genero', 'faixa_etaria', 'raca_cor', 'escolaridade']]
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11140 entries, 0 to 5569
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   ano           11140 non-null  int64   
 1   id_municipio  11140 non-null  int64   
 2   genero        11140 non-null  object  
 3   faixa_etaria  11101 non-null  category
 4   raca_cor      11140 non-null  object  
 5   escolaridade  11140 non-null  object  
dtypes: category(1), int64(2), object(3)
memory usage: 533.3+ KB


In [14]:
df1=df1.rename(columns={'raca_cor':'cor_raca'})

In [15]:
client = bigquery.Client()
dataset_ref = client.dataset('cargos_lideranca')

In [19]:
municipio=bd.read_sql("select sigla_uf, id_municipio, nome from `basedosdados.br_bd_diretorios_brasil.municipio`", billing_project_id="repositoriodedadosgpsp", from_file=False, reauth=False, use_bqstorage_api=False)

Downloading: 100%|██████████| 5570/5570 [00:00<00:00, 11073.83rows/s]


In [20]:
df1['id_municipio']=df1['id_municipio'].apply(str)

In [21]:
## Trazendo nomes de municipios

df1 = df1.merge(municipio, left_on='id_municipio', right_on='id_municipio')

In [24]:
df1=df1[['ano', 'sigla_uf', 'nome','id_municipio', 'genero', 'faixa_etaria', 'cor_raca',
       'escolaridade', ]]

In [25]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano da apuração daquele dado'),
 bigquery.SchemaField('sigla_uf','STRING',description='sigla uf do municipio'),
  bigquery.SchemaField('nome','STRING',description='Nome do município'),
 bigquery.SchemaField('id_municipio','STRING',description='Código do IBGE do município'),
 bigquery.SchemaField('genero','STRING',description='Gênero autodeclarado ou não'),
 bigquery.SchemaField('faixa_etaria','STRING',description='faixa etária da observação'),
 bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
 bigquery.SchemaField('escolaridade','STRING',description='Escolaridade da pessoa ou do vínculo observado com detalhamento na pós-graduação')

 ]


In [26]:
table_ref = dataset_ref.table('MUNIC_perfil_gestor_educacao_mapa')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df1,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=fa24d1ca-8051-433a-af14-15afd935efa3>